# CAPM Calculations with Exponential Weights #

### CAPM Model using Weighted Linear Regression Model ###

In [12]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm

# Pretty Notation
from IPython.display import display, Math

In [13]:
# Create the Weights function
def wexp(N, half_life):
    c = np.log(0.5)/half_life
    n = np.array(range(N))
    w = np.exp(c*n)
    return w/np.sum(w)

# Create the CAPM 
def CAPM(
    stock_prices: pd.Series, 
    benchmark_prices: pd.Series, 
    risk_free_rate: pd.Series, 
    window: int = 252,
    WLS: bool = False,
    flip: bool = False,
):
    """*+ nñ-0..
    Computes the rolling beta of a stock using Weighted Least Squares (WLS) regression.
    
    Parameters:
    stock_prices (pd.Series): Time series of stock prices.
    benchmark_prices (pd.Series): Time series of benchmark prices.
    risk_free_rate (pd.Series): Time series of annualized risk-free rate.
    window (int): Rolling window size for regression.
    decay (float): Decay factor for exponential weighting (0 < decay < 1).
    
    Returns:
    pd.Series: Rolling beta of the stock.
    """

    # Align time series to the same date range
    common_index = stock_prices.index.intersection(benchmark_prices.index).intersection(risk_free_rate.index)
    stock_prices = stock_prices.loc[common_index]
    benchmark_prices = benchmark_prices.loc[common_index]
    risk_free_rate = risk_free_rate.loc[common_index]
    
    # Compute daily returns
    stock_returns = stock_prices.pct_change(1)
    benchmark_returns = benchmark_prices.pct_change(1)
    risk_free_daily = (((1 + (risk_free_rate.div(100)))**(1/360)) - 1)  # Convert annual rate to daily
    
    # Excess returns
    excess_stock = stock_returns - risk_free_daily
    excess_benchmark = benchmark_returns - risk_free_daily

    alphas, betas = [], []
    p_values_alpha, p_values_beta = [], []
    
    for t in range(window, len(stock_returns)):
        X = excess_benchmark.iloc[t-window:t]
        y = excess_stock.iloc[t-window:t]
        
        if X.isnull().any() or y.isnull().any():
            continue

        if WLS:
            
            # Create weights with exponential decay
            weights = window * wexp(window, window/2)
            weights = np.flip(weights) if flip else weights
            
            # Fit WLS regression
            model = sm.WLS(y, sm.add_constant(X), weights=weights, missing='drop').fit()

        else:

            # Fit OLS regression
            model = sm.OLS(y, sm.add_constant(X), missing='drop').fit()

        # Avoid KeyError by checking if params exist
        params = model.params
        pvalues = model.pvalues
        
        alphas.append(params.iloc[0])
        betas.append(params.iloc[1])
        p_values_alpha.append(pvalues.iloc[0])
        p_values_beta.append(pvalues.iloc[1])
            
    parameters = pd.DataFrame({
        'alpha': alphas,
        'beta': betas,
        'p_value_alpha': p_values_alpha,
        'p_value_beta': p_values_beta
    }, index=stock_returns.index[window+1:])
    
    return parameters

In [14]:
# Get the important data for the Risk Free Rate

rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)
rfr.dropna(inplace = True)

rfr

In [15]:
# Get the important data for the S&P500

sp500 = pd.read_csv(r"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

sp500

In [16]:
# Stock Data
ticker = 'MSFT'

df_stock = pd.read_csv(rf"..\stocks\{ticker}.csv")
df_stock = df_stock.set_index('Date')
df_stock.index = pd.to_datetime(df_stock.index)

df_stock

In [17]:
# Calculate the Betas using WLS

betas_wls_without_flip = CAPM(
    df_stock['Adjusted_close'],
    sp500['sp_500'],
    rfr['risk_free_rate'],
    WLS = True,
    flip = False
)

betas_wls_without_flip

In [18]:
# Calculate the Betas using WLS

betas_wls_with_flip = CAPM(
    df_stock['Adjusted_close'],
    sp500['sp_500'],
    rfr['risk_free_rate'],
    WLS = True,
    flip = True
)

betas_wls_with_flip

In [19]:
# Calculate the Betas using OLS

betas_ols = CAPM(
    df_stock['Adjusted_close'],
    sp500['sp_500'],
    rfr['risk_free_rate'],
    WLS = False,
)

betas_ols

In [20]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_wls_without_flip['beta'], label='WLS Beta (with no flip)', color='orange', alpha=0.7)
plt.plot(betas_wls_with_flip['beta'], label='WLS Beta (with flip)', color='green', alpha=0.7)
plt.plot(betas_ols['beta'], label='OLS Beta', color='blue', alpha=0.7)
plt.axhline(y=1, color='black', linestyle='dashed')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Beta')
plt.legend()

# Show
plt.show()

In [21]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_wls_without_flip['alpha'], label='WLS Alpha (with no flip)', color='orange', alpha=0.7)
plt.plot(betas_wls_with_flip['alpha'], label='WLS Alpha (with flip)', color='green', alpha=0.7)
plt.plot(betas_ols['alpha'], label='OLS Alpha', color='blue', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Alpha Time Series')
plt.xlabel('Time')
plt.ylabel('Alpha')
plt.legend()

# Show
plt.show()

In [22]:
# Create Plot

weights = 252 * wexp(252, 126)

plt.figure(figsize=(10, 6))
plt.plot(weights, label='Weights', color='black', alpha=0.7)

# Config
plt.title('Weights (no flip) Graph')
plt.xlabel('Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.show()

In [23]:
# Let us make an example
weights = 252 * wexp(252, 126)
x = np.arange(1, 253) # A Trend Series

a_case = x.T @ weights
b_case = x.T @ np.flip(weights)

if a_case < b_case:
    print("Weights with no flip are biased to the future")
elif a_case > b_case:
    print("Weights with flip are biased to the past")
else:
    print("Both Weights are equal")

In [24]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_wls_without_flip['beta'].rolling(window=126).std(), label='WLS Beta (with no flip)', color='orange', alpha=0.7)
plt.plot(betas_wls_with_flip['beta'].rolling(window=126).std(), label='WLS Beta (with flip)', color='green', alpha=0.7)
plt.plot(betas_ols['beta'].rolling(window=126).std(), label='OLS Beta', color='blue', alpha=0.7)

# Config
plt.title('Rolling Standard Deviations Time Series')
plt.xlabel('Time')
plt.ylabel('Stds')
plt.legend()

# Show
plt.show()

In [25]:
# Calculate another kind of weights
lambda_decay = 0.97  
window = 252 

# Weights 
weights_alt = np.array([lambda_decay ** (window - t) for t in range(1, window + 1)])

In [26]:
plt.figure(figsize=(10, 6))
plt.plot(weights_alt, label='Weights', color='black', alpha=0.7)

# Config
plt.title('Weights (no flip) Graph')
plt.xlabel('Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.show()

Concluding that indeed we have to use the weights with flip